In [50]:
import pandas as pd
import numpy as np

In [51]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [52]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [53]:
from sklearn.model_selection import train_test_split

In [54]:
data = df[['o3']]

In [55]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [57]:
scaled = scaled.reshape((19706,))

In [58]:
scaled.shape

(19706,)

In [59]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [60]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [61]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [64]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 166, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 5312)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 168)                 │         892,584 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 892,712 (3.41 MB)

 Trainable params: 892,712 (3.41 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - loss: 0.0114 - root_mean_squared_error: 0.1034 - val_loss: 0.0067 - val_root_mean_squared_error: 0.0817 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - loss: 0.0063 - root_mean_squared_error: 0.0791 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0779 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - loss: 0.0060 - root_mean_squared_error: 0.0772 - val_loss: 0.0060 - val_root_mean_squared_error: 0.0774 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.0058 - root_mean_squared_error: 0.0760 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0766 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.0057 - root_mean_squared_error: 0.0753 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0768 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.0055 - root_mean_squa

In [65]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step


In [66]:
poll = np.array(df["o3"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [67]:
from sklearn.metrics import mean_squared_error

In [68]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 3.9823616


In [80]:
y_test_true.shape

(3874, 168)

In [81]:
testPredict.shape

(3874, 168)

In [85]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [86]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 1.0562
RMSE for segment 2: 1.7862
RMSE for segment 3: 2.4040
RMSE for segment 4: 2.7744
RMSE for segment 5: 3.0317
RMSE for segment 6: 3.1949
RMSE for segment 7: 3.2753
RMSE for segment 8: 3.3024
RMSE for segment 9: 3.3276
RMSE for segment 10: 3.3742
RMSE for segment 11: 3.3946
RMSE for segment 12: 3.4236
RMSE for segment 13: 3.4490
RMSE for segment 14: 3.4372
RMSE for segment 15: 3.4242
RMSE for segment 16: 3.4278
RMSE for segment 17: 3.4470
RMSE for segment 18: 3.5033
RMSE for segment 19: 3.5237
RMSE for segment 20: 3.5165
RMSE for segment 21: 3.5177
RMSE for segment 22: 3.5028
RMSE for segment 23: 3.4694
RMSE for segment 24: 3.4737
RMSE for segment 25: 3.5404
RMSE for segment 26: 3.5926
RMSE for segment 27: 3.6738
RMSE for segment 28: 3.7426
RMSE for segment 29: 3.7835
RMSE for segment 30: 3.8060
RMSE for segment 31: 3.8188
RMSE for segment 32: 3.8019
RMSE for segment 33: 3.7823
RMSE for segment 34: 3.7784
RMSE for segment 35: 3.8119
RMSE for segment 36: 3.8929
R

In [69]:
forecast

array([[0.16953434, 0.17763668, 0.18120164, ..., 0.16536057, 0.16864261,
        0.17170659],
       [0.5400383 , 0.53757674, 0.52591765, ..., 0.5442076 , 0.54172206,
        0.5271237 ],
       [0.32347432, 0.38298452, 0.44327778, ..., 0.17480955, 0.21052802,
        0.2479272 ],
       ...,
       [0.6621613 , 0.5748078 , 0.49260354, ..., 0.43986702, 0.4434381 ,
        0.4561344 ],
       [0.07048061, 0.10328917, 0.30267644, ..., 0.04588714, 0.04144385,
        0.06681569],
       [0.18922864, 0.16196665, 0.14003727, ..., 0.34463573, 0.28333074,
        0.2229071 ]], dtype=float32)

In [70]:
testPredict

array([[ 87.49779 ,  87.93814 ,  88.13189 , ...,  87.27095 ,  87.449326,
         87.615845],
       [107.634026, 107.500244, 106.86659 , ..., 107.86062 , 107.72553 ,
        106.93214 ],
       [ 95.86415 ,  99.09843 , 102.37526 , ...,  87.784485,  89.72572 ,
         91.7583  ],
       ...,
       [114.271194, 109.52369 , 105.05603 , ..., 102.189896, 102.38397 ,
        103.074   ],
       [ 82.114395,  83.897484,  94.733826, ...,  80.77778 ,  80.5363  ,
         81.915215],
       [ 88.56814 ,  87.0865  ,  85.89468 , ...,  97.01424 ,  93.68242 ,
         90.398506]], dtype=float32)

In [71]:
y_test_true

array([[ 88.52845 ,  86.56636 ,  84.58691 , ...,  87.13936 ,  87.781815,
         88.406906],
       [108.32301 , 107.454834, 106.58665 , ..., 114.9212  , 112.95911 ,
        110.97965 ],
       [ 95.99483 ,  99.64119 , 103.28755 , ...,  87.95545 ,  89.44872 ,
         90.959366],
       ...,
       [113.87938 , 108.49665 , 103.11392 , ...,  91.27191 ,  96.30737 ,
        101.32546 ],
       [ 81.46145 ,  82.972084,  93.56391 , ...,  78.40544 ,  78.457535,
         80.02026 ],
       [ 88.58054 ,  86.913635,  85.229355, ..., 100.11001 ,  96.04691 ,
         92.00118 ]], dtype=float32)

In [72]:
last_input = X_test[-1]

In [73]:
last_input.shape

(168, 1)

In [74]:
last_input = last_input.reshape(168,)

In [77]:
# predictions = []
# for _ in range(168):
#     # Predict the next value
#     print(last_input)
predicted_value = model.predict(last_input.reshape(1, 168, 1))[0]
predicted_value = predicted_value
predicted_value = predicted_value * stdop + meanop
    # predictions.append(predicted_value)
    
#     # Update the input sequence for the next iteration
#     last_input = np.append(last_input[1:], predicted_value)
# print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


In [78]:
predicted_value

array([ 88.56814 ,  87.0865  ,  85.89468 ,  85.56283 ,  85.12053 ,
        84.926125,  84.82616 ,  84.900276,  85.012024,  86.242004,
        87.66179 ,  89.064674,  93.91494 ,  98.99813 , 103.95776 ,
       104.310104, 104.34928 , 104.4447  , 102.89404 , 101.59718 ,
        99.94553 ,  96.863625,  93.75132 ,  90.66343 ,  89.166595,
        88.023994,  86.89277 ,  86.74036 ,  86.463684,  86.19579 ,
        86.0873  ,  85.94686 ,  85.92076 ,  86.63535 ,  87.637215,
        88.65329 ,  93.195015,  98.06164 , 103.030815, 104.152336,
       104.91652 , 105.34003 , 103.85257 , 102.05101 , 100.2362  ,
        96.88373 ,  93.53139 ,  90.27503 ,  88.84191 ,  87.40513 ,
        86.02289 ,  85.74353 ,  85.2294  ,  85.013855,  84.87958 ,
        85.02116 ,  85.15712 ,  86.54288 ,  88.187225,  89.803764,
        94.41786 ,  99.27209 , 104.060486, 104.77241 , 105.30119 ,
       105.5344  , 103.812996, 101.92653 ,  99.97254 ,  96.67348 ,
        93.36318 ,  90.06612 ,  88.883865,  87.41988 ,  86.257

In [35]:
y_test[-168:].ravel() * stdop + meanop

array([30.597725, 26.346886, 28.059309, 31.56474 , 28.663692, 29.550125,
       30.980501, 25.601479, 27.394485, 29.328516, 29.932901, 28.583109,
       27.172878, 28.139893, 31.504303, 29.932901, 25.601479, 26.346886,
       26.125278, 29.187494, 27.53551 , 28.663692, 30.899918, 27.253462,
       29.630709, 25.682062, 30.013485, 27.918285, 26.649078, 28.059309,
       25.299286, 29.550125, 26.870686, 28.804716, 26.2663  , 28.361502,
       26.2663  , 31.806494, 25.682062, 25.762648, 28.804716, 25.984255,
       28.663692, 26.346886, 26.649078, 28.804716, 24.553877, 26.508055,
       29.187494, 25.823086, 28.885302, 30.456701, 30.235092, 28.744278,
       24.715046, 29.187494, 25.238846, 29.630709, 27.47507 , 30.376118,
       29.711294, 24.553877, 31.423717, 29.106909, 29.026325, 25.460455,
       28.139893, 24.775486, 30.980501, 27.01171 , 27.696678, 24.997093,
       25.460455, 27.777262, 29.106909, 27.8377  , 28.965885, 28.059309,
       29.550125, 29.932901, 28.583109, 30.013485, 

In [79]:
model.save("o3_1dcnn.keras")